# Naive Bayes

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [1]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('emails/ham', 'ham'))


Let's have a look at that DataFrame:

In [2]:
data.head()

,message,class
emails/spam/00249.5f45607c1bffe89f60ba1ec9f878039a,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",spam
emails/spam/00373.ebe8670ac56b04125c25100a36ab0510,ATTENTION: This is a MUST for ALL Computer Use...,spam
emails/spam/00214.1367039e50dc6b7adb0f2aa8aba83216,This is a multi-part message in MIME format.\n...,spam
emails/spam/00210.050ffd105bd4e006771ee63cabc59978,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,spam
emails/spam/00033.9babb58d9298daa2963d4f514193d7d6,This is the bottom line. If you can GIVE AWAY...,spam


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [3]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [13]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?", "Hello, Bob, how are you?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham', 'ham'], dtype='<U4')

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.

In [7]:
df = DataFrame({'message': [], 'class': []})

df = df.append(dataFrameFromDirectory('emails/spam', 'spam'))
df = df.append(dataFrameFromDirectory('emails/ham', 'ham'))

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.2)

In [9]:
vectorizer2 = CountVectorizer()
counts2 = vectorizer2.fit_transform(df['message'].values)

classifier2 = MultinomialNB()
targets2 = df['class'].values
classifier2.fit(counts2, targets2)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
test_counts = vectorizer2.transform(test.message)
predictions2 = classifier2.predict(test_counts)
predictions2

array(['ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham',
       'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham',
       'spam', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'spam',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
    

In [42]:
test['class'][3]

'ham'

In [55]:
def accuracy(prediction, test, column, column2):
    count = 0
    details = {}
    detailed_content = {}
    for i in test[column]:
        if i not in details:
            details[i] = 0
            detailed_content[i] = []
    n = len(prediction)
    
    for i in range(n):
        if prediction[i] == test[column][i]:
            count += 1
        else:
            details[test[column][i]] += 1
            detailed_content[test[column][i]].append(test[column2][i])
            
    print('accuracy: ' + str(100 *(count/ n)) + "%\n")
    print("Following types are wrongly predicted, count:")
    
    for i in details:
        print(i + ": " + str(details[i]))
    print("\n<Detailed contents>")
    
    for i in detailed_content:
        print(i + ": ")
        if len(detailed_content[i]) == 0:
            print("None")
        else:
            print(DataFrame(detailed_content[i]))
        print("\n")
    
    
accuracy(predictions2, test, 'class', 'message')

accuracy: 97.83333333333334%

Following types are wrongly predicted, count:
ham: 0
spam: 13

<Detailed contents>
ham: 
None


spam: 
                                                    0
0   Hello, my name is Kelly, I am an 18 year old s...
1   ================This Is Definitely the Answer ...
2   NIGERIA ELECTIRC POWER AUTHORITY \n\nFEDERAL S...
3    REQUEST FOR URGENT BUSINESS ASSISTANCE\n\n---...
4   \n\n\n\nsocial\n\n\n\n\n\n\n\nOn January 1st 2...
5   Hi Job Seeker,\n\n\n\nWhen you create a FREE M...
6   \n\n                         Foreword\n\n\n\nA...
7   Are you tired of spending a fortune on printer...
8   \n\n>>From the desk of: DR. SAMUEL EBOKA.\n\nT...
9   ATTN:\n\n\n\nI am Edward Mulete JR. the son of...
10  Dear friend,\n\n\n\nI am Mrs. Sese-seko widow ...
11  Greetings! \n\n\n\nYou are receiving this lett...
12  Hi -\n\n\n\n( http://club.4tfox.com )\n\n\n\nY...


